In [ ]:
# parameter cell
DEVICES = "1,2"
FROM = "2019-01-01"
TO = "2020-01-01"
COUNTRY = "all"
ORDER = 1  # will be pass to param order in biLouvian call

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
import networkx as nx

tqdm.pandas()

In [ ]:
df = pd.read_parquet("../data/gen/preprocessed_2023-08-28T11-09-39.parquet")
df.info()

In [ ]:
df.dropna(axis=0, how="all", inplace=True)
print(len(df))
df.tail(2)

In [ ]:
if COUNTRY.lower() != "all":
    df = df[df["geocode_country_name"] == COUNTRY]
len(df)

In [ ]:
df["Year"] = df["fundraisingDate"].dt.year
df.head()

calculate the number of k-consecutive years that each lender invest in

In [ ]:
lender_year = df.groupby("lender_id").agg({"Year": set})

In [ ]:
lender_year["count"] = lender_year["Year"].apply(len)

In [ ]:
lender_year.sort_values("count", ascending=False, inplace=True)
lender_year

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.histplot(data=lender_year, x="count", bins=lender_year["count"].nunique())

# Add value labels to each bin
for p in ax.patches:
    ax.annotate(f"{p.get_height()}", (p.get_x() + p.get_width() / 2, p.get_height()), ha="center", va="bottom")

plt.title("Histogram of Lenders vs Years")
plt.show()

In [ ]:
# Function to find the longest consecutive sequence in a list
def find_longest_consecutive_years(years) -> int:
    longest_sequence = []
    current_sequence = []
    for year in sorted(years):
        if not current_sequence or year == current_sequence[-1] + 1:
            current_sequence.append(year)
        else:
            if len(current_sequence) > len(longest_sequence):
                longest_sequence = current_sequence
            current_sequence = [year]
    if len(current_sequence) > len(longest_sequence):
        longest_sequence = current_sequence
    return len(longest_sequence)


find_longest_consecutive_years([1, 3, 4, 5, 2, 8, 9])  # 1, 2, 3, 4, 5

In [ ]:
# Convert the set of years to a list for each lender
lender_year["Year"] = lender_year["Year"].apply(list)

# Apply the function to find the longest consecutive years for each lender
lender_year["longest_consecutive_years"] = lender_year["Year"].apply(find_longest_consecutive_years)

# Print the lender_year dataframe with the longest consecutive years
lender_year.head()

In [ ]:
lender_year["longest_consecutive_years"].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.histplot(
    data=lender_year, x="longest_consecutive_years", bins=lender_year["longest_consecutive_years"].nunique()
)

# Add value labels to each bin
for p in ax.patches:
    ax.annotate(f"{p.get_height()}", (p.get_x() + p.get_width() / 2, p.get_height()), ha="center", va="bottom")

plt.title("Histogram of Lenders vs Years")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.histplot(
    data=lender_year, x="longest_consecutive_years", bins=lender_year["longest_consecutive_years"].nunique()
)

# Add value labels to each bin
for p in ax.patches:
    ax.annotate(f"{p.get_height()}", (p.get_x() + p.get_width() / 2, p.get_height()), ha="center", va="bottom")

plt.title("Histogram of Lenders vs Years")
plt.yscale("log")  # Set y-axis to log scale
plt.show()